In [8]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

# 策略

In [11]:
class Strategy_comp_mul(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('p_stake', 100),
        ('buy_signal', 'b'),
        ('sell_signal', 'b'),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('rsi_short', 6),
        ('rsi_long', 12),
        ('wave_period', 7),
        ('wr', 14),
        
        # ATR params
        ('using_atr', False),
        ('atr_period', 14),
        ('atr_percent', 1),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):

        # Record ATR Initial
        self.atr_initial = self.broker.get_cash()
        self.inds = dict()
        self.data_tracker = None

        for d in self.datas:

            self.inds[d] = dict()
            
            # Keep references to lines in the data[0] dataseries
            self.inds[d]['dataclose'] = d.close
            self.inds[d]['datahigh'] = d.high
            self.inds[d]['datalow'] = d.low

            # Keep references to executed order & price
            self.inds[d]['buyprice'] = 0
            self.inds[d]['sellprice'] = 0
            self.inds[d]['order'] = None
                
            # Add indicators
            ## Wave indicator
            ### MovingAverageSimple indicator
            self.inds[d]['sma'] = bt.indicators.SimpleMovingAverage(d, period=self.params.wave_period)
            self.inds[d]['wave_buy'] = bt.And(self.inds[d]['sma'](0) > self.inds[d]['sma'](-1), 
                                              self.inds[d]['sma'](-1) < self.inds[d]['sma'](-2), 
                                              self.inds[d]['sma'](-2) < self.inds[d]['sma'](-3))

            self.inds[d]['wave_sell'] = bt.And(self.inds[d]['sma'](0) < self.inds[d]['sma'](-1), 
                                               self.inds[d]['sma'](-1) > self.inds[d]['sma'](-2), 
                                               self.inds[d]['sma'](-2) > self.inds[d]['sma'](-3))

            ## MACD indicator
            self.inds[d]['macdhisto'] = bt.indicators.MACDHisto(d)
            self.inds[d]['macd_buy'] = bt.And(self.inds[d]['macdhisto'](0) > 0, self.inds[d]['macdhisto'](-1) < 0)
            self.inds[d]['macd_sell'] = bt.And(self.inds[d]['macdhisto'](0) < 0, self.inds[d]['macdhisto'](-1) > 0)

            ## Bollinger Bands indicator
            self.inds[d]['bolling'] = bt.indicators.BollingerBands(d, 
                                                        period=self.params.bollinger_per, 
                                                        devfactor=self.params.bollinger_dev)
            self.inds[d]['bolling_buy'] = bt.Or(bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bolling'].top(0), 
                                                       self.inds[d]['dataclose'](-1) < self.inds[d]['bolling'].top(-1)), 
                                                bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bolling'].bot(0), 
                                                       self.inds[d]['dataclose'](-1) < self.inds[d]['bolling'].bot(-1)))
            self.inds[d]['bolling_sell'] = bt.Or(bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bolling'].top(0), 
                                                        self.inds[d]['dataclose'](-1) > self.inds[d]['bolling'].top(-1)), 
                                                 bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bolling'].bot(0), 
                                                        self.inds[d]['dataclose'](-1) > self.inds[d]['bolling'].bot(-1)))

            ## RSI indicator
            self.inds[d]['rsi_s'] = bt.indicators.RelativeStrengthIndex(d, period=self.params.rsi_short, safediv=True)
            self.inds[d]['rsi_l'] = bt.indicators.RelativeStrengthIndex(d, period=self.params.rsi_long, safediv=True)
            self.inds[d]['rsi_buy'] = bt.And(self.inds[d]['rsi_s'](0) > self.inds[d]['rsi_l'](0), 
                                             self.inds[d]['rsi_s'](-1) < self.inds[d]['rsi_l'](-1))
            self.inds[d]['rsi_sell'] = bt.And(self.inds[d]['rsi_s'](0) < self.inds[d]['rsi_l'](0), 
                                              self.inds[d]['rsi_s'](-1) > self.inds[d]['rsi_l'](-1))

            ## WilliamR indicator
            self.inds[d]['wr'] = bt.indicators.WilliamsR(d, period=self.params.wr)
            self.inds[d]['wr_buy'] = self.inds[d]['wr'] < -80
            self.inds[d]['wr_sell'] = self.inds[d]['wr'] > -20
        
            ## ATR indicator
            self.inds[d]['tr'] = bt.indicators.Max((self.inds[d]['datahigh'](0) - self.inds[d]['datalow'](0)), 
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datahigh'](0)),
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datalow'](0)))
            self.inds[d]['atr'] = bt.indicators.SimpleMovingAverage(self.inds[d]['tr'], period=self.params.atr_period)





    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.inds[self.data_tracker]['wave_buy']:
                    self.inds[self.data_tracker]['buyprice']
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:  # Sell
                self.inds[self.data_tracker]['sellprice'] = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled]:
            self.log('Order Canceled')

        elif order.status in [order.Margin]:
            self.log('Order Margin')

        elif order.status in [order.Rejected]:
            self.log('Order Rejected')

        # Write down: no pending order
        #self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    

    def next(self):

        buy_list = []
        sell_list = []

        for d in self.datas:
        # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.inds[d]['order']:
                return
            self.data_tracker = d
            buy_indicator = (self.params.buy_signal == 'b' and self.inds[d]['bolling_buy']) or \
                            (self.params.buy_signal == 'm' and self.inds[d]['macd_buy']) or \
                            (self.params.buy_signal == 'w' and self.inds[d]['wave_buy'] and self.inds[d]['dataclose'][0] > self.inds[d]['buyprice']) or \
                            (self.params.buy_signal == 'r' and self.inds[d]['rsi_buy']) or \
                            (self.params.buy_signal == 'wr' and self.inds[d]['wr_buy'])
            
            #buy_indicator = self.inds[d]['bolling_buy'] and \
            #                self.inds[d]['macd_buy'] and \
            #                (self.inds[d]['wave_buy'] and self.inds[d]['dataclose'][0] > self.inds[d]['buyprice']) and \
            #                self.inds[d]['rsi_buy'] and \
            #                self.inds[d]['wr_buy']
                            
            sell_indicator = (self.params.sell_signal == 'b' and self.inds[d]['bolling_sell']) or \
                            (self.params.sell_signal == 'm' and self.inds[d]['macd_sell']) or \
                            (self.params.sell_signal == 'w' and self.inds[d]['wave_sell']) or \
                            (self.params.sell_signal == 'r' and self.inds[d]['rsi_sell']) or \
                            (self.params.sell_signal == 'wr' and self.inds[d]['wr_sell'])
            
            #sell_indicator = self.inds[d]['bolling_sell'] or \
            #                 self.inds[d]['macd_sell'] or \
            #                 self.inds[d]['wave_sell'] or \
            #                 self.inds[d]['rsi_sell'] or \
            #                 self.inds[d]['wr_sell']

            if sell_indicator and self.getposition(d).size > 0:
                sell_list.append((d, -1))
                #self.log('Sell CREATE(Close), Code: %s, Price: %.2f, Current Position: %i' % 
                #         (d._name, 
                #          self.inds[d]['dataclose'][0],
                #          self.getposition(d).size))
                #self.close(data=d, name=d._name)
                
            elif buy_indicator:
                if self.params.using_atr:
                    opt_pos = np.floor(self.atr_initial * self.params.atr_percent / self.params.p_stake / self.inds[d]['atr'][0])
                    if opt_pos*self.params.p_stake > 0 and self.getposition(d).size < opt_pos*self.params.p_stake:
                        buy_list.append((d, 
                                        opt_pos*self.params.p_stake-self.getposition(d).size, 
                                        (opt_pos*self.params.p_stake-self.getposition(d).size)*self.inds[d]['dataclose'][0]))
                        #self.log('BUY CREATE, Code: %s, Price: %.2f, Lots: %i, Current Position: %i' % 
                        #        (d._name, 
                        #         self.inds[d]['dataclose'][0], 
                        #         opt_pos*self.params.p_stake-self.getposition(d).size,
                        #         self.getposition(d).size))
                        #self.buy(data=d, size=opt_pos*self.params.p_stake-self.getposition(d).size, name=d._name)
                    
                    elif opt_pos*self.params.p_stake > 0 and self.getposition(d).size > opt_pos*self.params.p_stake:
                        sell_list.append((d, 
                        self.getposition(d).size-opt_pos*self.params.p_stake))
                        #self.log('SELL CREATE, Code: %s, Price: %.2f, Lots: %i, Current Position: %i' % 
                        #        (d._name, 
                        #         self.inds[d]['dataclose'][0], 
                        #         self.getposition(d).size-opt_pos*self.params.p_stake,
                        #         self.getposition(d).size))
                        #self.sell(data=d, size=self.getposition(d).size-opt_pos*self.params.p_stake, name=d._name)
                else:
                    buy_list.append((d, 
                                     opt_pos*self.params.p_stake, 
                                     opt_pos*self.params.p_stake*self.inds[d]['dataclose'][0]))
                    #self.log('BUY CREATE, Code: %s, Price: %.2f, Lots: %i, Current Position: %i' % 
                    #        (d._name, 
                    #         self.inds[d]['dataclose'][0],
                    #         self.params.p_stake,
                    #         self.getposition(d).size))
                    #self.buy(data=d, size=self.params.p_stake, name=d._name)
            
        for s_order in sell_list:
            self.log('Sell CREATE, Code: %s, Price: %.2f, Lots:%i, Current Position: %i' % 
                        (s_order[0]._name, 
                        self.inds[s_order[0]]['dataclose'][0],
                        s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size,
                        self.getposition(s_order[0]).size))
            self.sell(data=s_order[0], 
                        size=s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size, 
                        name=s_order[0]._name)
        
        #buy_list = sorted(buy_list, key=lambda x: x[2], reverse=False)
        random.shuffle(buy_list)
        for b_order in buy_list:
            self.log('BUY CREATE, Code: %s, Price: %.2f, Lots: %i, Current Position: %i' % 
                        (b_order[0]._name, 
                        self.inds[b_order[0]]['dataclose'][0], 
                        b_order[1],
                        self.getposition(b_order[0]).size))
            self.buy(data=b_order[0], size=b_order[1], name=b_order[0]._name)


    def stop(self):
        self.log('Ending Position:%i. Ending Value:%.2f.' %
                (1,
                self.cerebro.broker.getvalue()), doprint=False)

# 全市场选股

## 数据下载

In [3]:
s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)

#s_date = datetime.datetime.now() - datetime.timedelta(days = 365)
#e_date = datetime.datetime.now()

stock_index = '399300.SZ'

# Download Data
utilsJ.index_to_csv_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                            stock_index, 0.2, s_date, e_date, fpath='.\\Data\\2020\\')

In [23]:
profit_list = []

# 904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157
# 74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)

stock_index = '399300.SZ'
pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
index_list = np.unique(pro.index_weight(index_code=stock_index,
                                        start_date=s_date.strftime('%Y%m%d'),
                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()

buy_s = ['b','m','w', 'r', 'wr']
sell_s = ['b','m','w', 'r', 'wr']
random.seed(10086)
for s in product(*[buy_s, sell_s]):

    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(Strategy_comp_mul, printlog=False, using_atr=True, atr_percent=0.01, buy_signal = s[0], sell_signal = s[1]) 

    for s_code in index_list:
        df = pd.read_csv('.\\Data\\2020\\' + s_code + '.csv')
        df.index=pd.to_datetime(df.trade_date)
        data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

        # Add the index Data Feed to Cerebo
        cerebro.adddata(data, name = s_code)
    
    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(1000000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value - start_value) / start_value * 100))
    profit_list.append((s, (final_value - start_value) / start_value * 100))

(b, b) Net Profit: 13.95%
(b, m) Net Profit: 1.61%
(b, w) Net Profit: -2.69%
(b, r) Net Profit: 14.49%
(b, wr) Net Profit: 7.12%
(m, b) Net Profit: 5.05%
(m, m) Net Profit: 8.03%
(m, w) Net Profit: -13.08%
(m, r) Net Profit: -6.21%
(m, wr) Net Profit: 37.92%
(w, b) Net Profit: 7.29%
(w, m) Net Profit: 16.89%
(w, w) Net Profit: -6.62%
(w, r) Net Profit: 16.45%
(w, wr) Net Profit: 4.36%
(r, b) Net Profit: 16.16%
(r, m) Net Profit: 12.25%
(r, w) Net Profit: -24.00%
(r, r) Net Profit: 7.10%
(r, wr) Net Profit: -5.73%
(wr, b) Net Profit: -2.23%
(wr, m) Net Profit: 6.31%
(wr, w) Net Profit: -2.08%
(wr, r) Net Profit: 1.36%
(wr, wr) Net Profit: 5.80%


In [24]:
random.seed(10086)

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)

stock_index = '399300.SZ'
pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
index_list = np.unique(pro.index_weight(index_code=stock_index,
                                        start_date=s_date.strftime('%Y%m%d'),
                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()
profit = 0
for i in range(20):

    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(Strategy_comp_mul, printlog=True, using_atr=True, atr_percent=0.01, buy_signal = 'm', sell_signal = 'wr') 

    for s_code in index_list:
        df = pd.read_csv('.\\Data\\2020\\' + s_code + '.csv')
        df.index=pd.to_datetime(df.trade_date)
        data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

        # Add the index Data Feed to Cerebo
        cerebro.adddata(data, name = s_code)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(500000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    profit += (final_value - start_value) / start_value * 100


2020-08-13: BUY CREATE, Code: 601186.SH, Price: 9.07, Lots: 21200, Current Position: 0
2020-08-13: BUY CREATE, Code: 600025.SH, Price: 3.86, Lots: 61900, Current Position: 0
2020-08-13: BUY CREATE, Code: 300070.SZ, Price: 9.27, Lots: 17400, Current Position: 0
2020-08-14: Order Margin
2020-08-14: BUY EXECUTED, Price:9.07, Lot:21200, Cash:68782.
2020-08-14: BUY EXECUTED, Price:3.86, Lot:61900, Cash:68782.
2020-08-14: Sell CREATE, Code: 600025.SH, Price: 3.92, Lots:61900, Current Position: 61900
2020-08-14: BUY CREATE, Code: 600928.SH, Price: 5.72, Lots: 33900, Current Position: 0
2020-08-14: BUY CREATE, Code: 600027.SH, Price: 3.81, Lots: 47200, Current Position: 0
2020-08-14: BUY CREATE, Code: 600050.SH, Price: 5.32, Lots: 51000, Current Position: 0
2020-08-14: BUY CREATE, Code: 600009.SH, Price: 75.15, Lots: 2600, Current Position: 0
2020-08-14: BUY CREATE, Code: 002230.SZ, Price: 38.45, Lots: 3900, Current Position: 0
2020-08-14: BUY CREATE, Code: 000671.SZ, Price: 7.30, Lots: 21400,